In [3]:
import numpy as np
import sympy as sp
from sympy import ntheory
from sympy.ntheory import continued_fraction_periodic
import math

In [4]:
def classRepresentatives(trace): # gives quadratic forms of given trace
    bound = math.floor(trace**2-4)
    quadraticForms = []
    i = 1
    while i**2 < bound:
        if not bound % (i**2) == 0:
            i = i + 1
            continue
        D = bound//(i**2)
        if isASquare(D):
            i = i + 1
            continue
        elif D%4 == 0:
            quadraticForms+=representatives(D)
        elif D%4 == 1:
            quadraticForms+=representatives(D)
        i = i + 1
    return [Q for Q in quadraticForms if tr(Q)==np.round(trace)]

def representatives(D):
    reps = []
    seenFracs = []
    for i in range(1,math.floor(np.sqrt(D))+1):
        B = -i
        if not (B**2-D)%4==0:
            continue
        for j in range(math.ceil((np.sqrt(D)+B)/2),math.floor((np.sqrt(D)-B)/2)+1):
            A = j
            if not ((B**2-D)//4)%A==0:
                continue
            C = ((B**2-D)//4)//A
            Q = [A,B,C]
            frac = ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0]
            if frac not in seenFracs and gcd(A,B,C)==1:
                seenFracs+=permutationsOf(frac)
                reps.append(Q)
            else:
                continue
    return reps

def permutationsOf(a):
    perms = []
    if len(a)%2==0:
        for n in range(0,len(a),2):
            perms.append(a[n:]+a[:n])
    else:
        for n in range(0,len(a)):
            perms.append(a[n:]+a[:n])
    return perms

def disc(Q):
    return Q[1]**2-4*Q[0]*Q[2]

def areEquivalent(Q,W):
    return ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0] in permutationsOf(ntheory.continued_fraction_periodic(-W[1], 2*W[0], disc(W))[0])

def equalsQuad(L,M):
    if len(L)!=len(M):
        return False
    for l in L:
        counter = 0
        for m in M:
            if areEquivalent(m,l):
                M.remove(m)
                counter+=1
        if counter !=1:
            return False
    return True

def classNum(D):
    return len(representatives(D))

def tr(Q):
    return np.trace(quadToMat(Q))

def isASquare(n):
    for i in range(2,n):
        if n==i**2:
            return True
    return False

def gcd(a,b,c):
    return np.gcd(a,np.gcd(b,c))

def matgcd(mat):
    return gcd(mat[1,0],mat[1,1]-mat[0,0],mat[0,1])

def quadToMat(Q):
    mat = np.identity(2)
    length = 0
    for x in ntheory.continued_fraction_periodic(-Q[1], 2*Q[0],disc(Q))[0]:
        mat = np.matmul(mat,[[x,1],[1,0]])
        length += 1
    if length % 2 == 1:
        mat = np.matmul(mat,mat)
    return np.array(mat).astype(int)

In [7]:
def se(ls):
    if len(ls)%2==1:
        ls = ls + ls
    output = []
    for i in range(len(ls)):
        if i%2==1: #if original cont'd frac is even length, making last one R/alternating is same as this (hopefully)
            for j in range(ls[i]):
                output.append(1)
        else:
            for j in range(ls[i]):
                output.append(0)
    return output

def recover2(ls): #sends a whole list to a cont'd fraction (not clear whether this is needed)
    output = []
    i=0
    while i<len(ls):
        counter=1
        while(i<len(ls)-1 and ls[i] == ls[i+1]):
            i+=1
            counter+=1
        i+=1
        output.append(counter)
    if (len(output)%2!=0):
        output[len(output)-1]+=output[0]
        output = output[1:]
    return output

def indices(ls): #positions of R's (R=1)
    return [i for i in range(len(ls)) if ls[i]==1]

def pairs(ls): #pairs of R's on which to split
    output = []
    for i in indices(ls):
        for j in indices(ls):
            if i==j:
                continue
            else:
                output.append((i,j))
    return output

def split(ls): #splits a cont'd fraction on every possible pair of R's into 2 groups of R's and L's (in 0's and 1's)
    output = []
    converted = se(ls)
    n = len(converted)
    splitPoints = pairs(converted)
    for i,j in splitPoints:
        temp = converted[i:]+converted[:i]
        temp1 = temp[1:(j-i)%n]
        temp2 = inv(temp[(j-i)%n+1:])
        output.append((temp1,temp2))
    return output

def printCon(ls): #R = 1, L = 0
    a = ''
    for i in ls:
        if i==1:
            a+= 'R'
        else:
            a+= 'L'
    print(a)

def printConLs(ls):
    for L in ls:
        printCon(L)

def getDict(trace,e):
    output = []
    fracs = []
    for t in range(1+2*e):
        tr = trace-e+t
        fracs += [ntheory.continued_fraction_periodic(-Q[1], 2*Q[0], disc(Q))[0] for Q in classRepresentatives(tr)]
    for f in fracs:
        output += split(f)
    return output

def inv(ls): #inverts a list of 0's and 1's (R's and L's)
    return list((np.array(ls[::-1])-1)**2)

def pairToFrac(a,b):
    out = a+[1]+b+[1]
    return recover2(out)

def getPants(trace, e): #this clearly doesn't work
    allPairs = getDict(trace,e)
    output,fracs = [],[]
    for s0,s1 in allPairs:
        temp1,temp2 = set([tuple(a[1]) for a in allPairs if a[0]==s0]), set([tuple(a[1]) for a in allPairs if a[0]==s1])
        s  = temp1.intersection(temp2)
        for s2 in s:
            s2 = list(s2)
            output.append([s0,s1,s2])
            fracs = [pairToFrac(s0,inv(s1)),pairToFrac(s1,inv(s2)),pairToFrac(s2,inv(s0))]
    return fracs

In [6]:
[(t,getPants(t,4)) for t in range(15,30)]

[(15, [[1, 17], [3, 3], [1, 1, 12, 2]]),
 (16, [[3, 6], [3, 1, 1, 1], [1, 6]]),
 (17, [[1, 19], [3, 5], [3, 1, 14, 2]]),
 (18, [[2, 10], [3, 5], [3, 1, 5, 3]]),
 (19, [[1, 21], [3, 5], [3, 1, 16, 2]]),
 (20, [[1, 22], [3, 5], [3, 1, 17, 2]]),
 (21, [[1, 7, 1, 1], [1, 15], [12, 1, 5, 2]]),
 (22, [[2, 12], [4, 6], [4, 1, 6, 3]]),
 (23, [[5, 5], [1, 24], [11, 1, 6, 1, 3, 1]]),
 (24, [[1, 26], [4, 6], [4, 1, 20, 2]]),
 (25, [[3, 9], [4, 6], [4, 1, 3, 4]]),
 (26, [[1, 6, 1, 2], [1, 22], [19, 1, 5, 2]]),
 (27, [[1, 29], [4, 6], [4, 1, 23, 2]]),
 (28, [[1, 30], [4, 6], [4, 1, 24, 2]]),
 (29, [[1, 31], [4, 6], [4, 1, 25, 2]])]